# David's Work

In [1]:
import pandas as pd

In [71]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix, confusion_matrix, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
!ls ../../data

features.csv
target.csv


In [3]:
df = pd.read_csv('../../data/features.csv')

In [4]:
target = pd.read_csv('../../data/target.csv')

In [5]:
print(df.shape)
print(target.shape)

(59400, 40)
(59400, 2)


In [6]:
pd.set_option('display.max_columns', None)

## Business Problem

- Be able to predict if waterwell is functional or not. Explore important metrics that can be used as things to make a water well last longer
- Focus on getting water to everyone

**Context**
- False-Positive - Predict functional well when its not (reduce)
- False-Negative - Predict non-functional when it is (reduce but not as important)
- Would rather say a well is not functional and is versus functional and is not.
- Put 'needs work' wells with non-functional wells. To turn into binary classification

**Evaluation**
- Accuracy - balances the two kinds of errors (but is impractical with imbalanced targets)
- **Precision** - helps reduce false positives
- Recall - helps reduce false negatives
- **F1-Score** - balances recall & precision (and is better than accuracy with imbalanced targets)
- **ROC-AUC** - helps focus on better probability outputs (makes sure our predicted probabilities are better)

## Data Exploration

- explore null values, reduce variables

In [7]:
df.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [8]:
target.head(2)

,id,status_group
0,69572,functional
1,8776,functional


In [9]:
data = df.merge(target, on=['id'])

In [10]:
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [12]:
data.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [13]:
data['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

**Get rid of types and keep groups**

In [14]:
data['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

In [15]:
data['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

**Drop scheme name, we could still use scheme mangement**

In [16]:
data['scheme_name'].value_counts()

K                             682
None                          644
Borehole                      546
Chalinze wate                 405
M                             400
                             ... 
Mtumbei mpopera                 1
Mashangwi                       1
Heka water supply               1
Mhembe                          1
Charles Mushi  Branch line      1
Name: scheme_name, Length: 2696, dtype: int64

In [17]:
data['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

**Drop funder**
- values seem irrelevant 


In [18]:
data['funder'].nunique()

1897

In [19]:
data['funder'].value_counts()

Government Of Tanzania        9084
Danida                        3114
Hesawa                        2202
Rwssp                         1374
World Bank                    1349
                              ... 
Karadea Ngo                      1
Paffect Mwanaindi                1
Muslimehefen International       1
Pwagu                            1
Hotels And Loggs Tz Ltd          1
Name: funder, Length: 1897, dtype: int64

In [20]:
data['installer'].nunique()

2145

**Possibly drop village and codes in favor of latitude and longitude**

In [21]:
data['subvillage'].value_counts()

Madukani      508
Shuleni       506
Majengo       502
Kati          373
Mtakuja       262
             ... 
Lukuju          1
Nyanghanga      1
Tamaya          1
Ajida           1
Bukenge         1
Name: subvillage, Length: 19287, dtype: int64

**DROP, NOT USEFUL**

In [22]:
data['num_private'].value_counts()

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

In [23]:
data['permit'].value_counts(normalize=True)

True     0.68955
False    0.31045
Name: permit, dtype: float64

**Probably keep type class for extraction**

In [24]:
data['extraction_type'].value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [25]:
data['extraction_type_class'].value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

In [26]:
data['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [50]:
location = data[['region',                       
'region_code',                  
'district_code',                
'lga', 
'ward']]

**Keep region drop everything else location wise**

In [51]:
[print(x,'\n*'*2, data[x].value_counts()) for x in location]

region 
*
* Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64
region_code 
*
* 11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64
district_code 
*
* 1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43

[None, None, None, None, None]

## Further exploring after dropping cols

### Cleaning Insights

- too many values in certain columns, a bunch of one-time values
- got rid of columns that shared same vales (extraction, codes, management)
- got rid of irrelevant data (id, permit, num_private)\
- installer/scheme management had lots of values with some being one-shots and lots of nulls

In [84]:
drop = ['id', 'waterpoint_type', 'scheme_name', 'funder', 
        'num_private', 'permit', 'extraction_type', 'extraction_type_group',
        'region_code','district_code','lga','ward', 'recorded_by', 'public_meeting', 
        'scheme_management', 'installer', 'payment', 'subvillage']

In [85]:
data.drop(columns=drop).head()

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type_group,status_group
0,6000.0,2011-03-14,1390,34.938093,-9.856322,none,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,functional
1,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
2,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe,functional
3,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe,non functional
4,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional


In [86]:
clean_data = data.drop(columns=drop)

In [87]:
clean_data.isna().sum()

amount_tsh               0
date_recorded            0
gps_height               0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
region                   0
population               0
construction_year        0
extraction_type_class    0
management               0
management_group         0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type_group    0
status_group             0
dtype: int64

In [77]:
data['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

**Making a new col indicating if a population is near well or not**

In [128]:
near_pop = []
for x in list(clean_data['population'].values):
    if x == 0:
        near_pop.append(False)
    else:
        near_pop.append(True)

In [129]:
clean_data['near_pop'] = near_pop

In [130]:
clean_data.head()

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type_group,status_group,near_pop
0,6000.0,2011-03-14,1390,34.938093,-9.856322,none,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,functional,True
1,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional,True
2,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe,functional,True
3,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe,non functional,True
4,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional,False


In [131]:
clean_data['wpt_name'].value_counts()

none                3563
Shuleni             1748
Zahanati             830
Msikitini            535
Kanisani             323
                    ... 
Kwa Akalama            1
Kilelema Primary       1
Kwa Mwanahila          1
Kwa Yoramu             1
Nyarunyomvi            1
Name: wpt_name, Length: 37400, dtype: int64

In [132]:
clean_data['basin'].value_counts()

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

In [133]:
clean_data[['water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class']].head(25)

,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class
0,soft,good,enough,enough,spring,spring,groundwater
1,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface
2,soft,good,enough,enough,dam,dam,surface
3,soft,good,dry,dry,machine dbh,borehole,groundwater
4,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface
5,salty,salty,enough,enough,other,other,unknown
6,soft,good,enough,enough,machine dbh,borehole,groundwater
7,milky,milky,enough,enough,shallow well,shallow well,groundwater
8,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater
9,soft,good,enough,enough,shallow well,shallow well,groundwater


In [134]:
clean_data['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [135]:
clean_data['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

In [136]:
clean_data['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [137]:
clean_data['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [138]:
clean_data['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

### Cleaning Insights

- too many values in wpt_name, a bunch of one-time values
- got rid of columns that shared same vales (quantity, source, etc)
- favored region for location data vs lat/long

In [139]:
clean_drop = ['wpt_name', 'water_quality', 'quantity', 'source', 'date_recorded', 'longitude', 'latitude']

## Cleaning up final data

In [140]:
final_data = clean_data.drop(columns=clean_drop)

In [141]:
for x in final_data:
    display(x,'******',final_data[x].value_counts())

'amount_tsh'

'******'

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

'gps_height'

'******'

...

'******'

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

'near_pop'

'******'

True     38019
False    21381
Name: near_pop, dtype: int64

In [142]:
final_data[['amount_tsh', 'gps_height', 'construction_year', 'population']]

,amount_tsh,gps_height,construction_year,population
0,6000.0,1390,1999,109
1,0.0,1399,2010,280
2,25.0,686,2009,250
3,0.0,263,1986,58
4,0.0,0,0,0
...,...,...,...,...
59395,10.0,1210,1999,125
59396,4700.0,1212,1996,56
59397,0.0,0,0,0
59398,0.0,0,0,0


### amount_tsh 
- Total static head (amount water available to waterpoint)
- The static head, the amount of water in a well when the well is "at rest" - that is, no one has pumped water out of the well for some time and the well has filled back up as much as it's going to
- Make indicator column if there is static water or not

### population
- population amount near water well
- make an indicator column if there is a population near or not

### gps_height
- Altitude of the well
- make column indicator if it above or below ground level or at ground level
- Water levels in wells are usually reported as depths below land surface, although the measuring point can be any convenient fixed place near the top of the well. In this drawing the measuring point is the top of the casing. The altitude of the measuring point is commonly recorded so that static water levels can also be reported as altitudes.

### contruction year
- year it was constructed 
- can probably make an indicator column if it has one or not
- furthermore could make insights on dates we do have (older wells, recent wells...)

**sources**


https://mbmggwic.mtech.edu/sqlserver/v11/help/welldesign.asp


https://inspectapedia.com/water/Well_Static_Head_Definition.php#:~:text=The%20static%20head%2C%20the%20amount,dry%20season%20of%20the%20year

In [143]:
final_data.head()

,amount_tsh,gps_height,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,quality_group,quantity_group,source_type,source_class,waterpoint_type_group,status_group,near_pop
0,6000.0,1390,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,good,enough,spring,groundwater,communal standpipe,functional,True
1,0.0,1399,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional,True
2,25.0,686,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,good,enough,dam,surface,communal standpipe,functional,True
3,0.0,263,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,good,dry,borehole,groundwater,communal standpipe,non functional,True
4,0.0,0,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional,False


In [155]:
final_data['gps_height'].value_counts()

 0       20438
-15         60
-16         55
-13         55
-20         52
         ...  
 2285        1
 2424        1
 2552        1
 2413        1
 2385        1
Name: gps_height, Length: 2428, dtype: int64

In [156]:
final_data[['gps_height', 'amount_tsh']]

,gps_height,amount_tsh
0,1390,6000.0
1,1399,0.0
2,686,25.0
3,263,0.0
4,0,0.0
...,...,...
59395,1210,10.0
59396,1212,4700.0
59397,0,0.0
59398,0,0.0


In [161]:
lvl = []
for x in list(final_data['gps_height'].values):
    if x == 0:
        lvl.append('level')
    elif x > 0:
        lvl.append('above')
    else:
        lvl.append('below')

In [162]:
final_data['level'] = lvl

In [167]:
stwater = []

for x in list(final_data['amount_tsh'].values):
    if x == 0:
        stwater.append(False)
    else:
        stwater.append(True)

In [168]:
final_data['has_static'] = stwater

In [170]:
year = []

for x in list(final_data['construction_year'].values):
    if x == 0:
        year.append(False)
    else:
        year.append(True)

In [171]:
final_data['has_year'] = year

In [173]:
final_data['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

**following our business problem**

In [177]:
final_data['status_group'] = final_data["status_group"].replace("functional needs repair", "non functional")

In [178]:
final_data['status_group'].value_counts()

functional        32259
non functional    27141
Name: status_group, dtype: int64

In [179]:
final_data.head()

,amount_tsh,gps_height,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,quality_group,quantity_group,source_type,source_class,waterpoint_type_group,status_group,near_pop,level,has_static,has_year
0,6000.0,1390,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,good,enough,spring,groundwater,communal standpipe,functional,True,above,True,True
1,0.0,1399,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional,True,above,False,True
2,25.0,686,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,good,enough,dam,surface,communal standpipe,functional,True,above,True,True
3,0.0,263,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,good,dry,borehole,groundwater,communal standpipe,non functional,True,above,False,True
4,0.0,0,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional,False,level,False,False


In [180]:
final_data['status_group'] = final_data["status_group"].replace("non functional", "non functional/needs repair")

In [181]:
final_data.head()

,amount_tsh,gps_height,basin,region,population,construction_year,extraction_type_class,management,management_group,payment_type,quality_group,quantity_group,source_type,source_class,waterpoint_type_group,status_group,near_pop,level,has_static,has_year
0,6000.0,1390,Lake Nyasa,Iringa,109,1999,gravity,vwc,user-group,annually,good,enough,spring,groundwater,communal standpipe,functional,True,above,True,True
1,0.0,1399,Lake Victoria,Mara,280,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional,True,above,False,True
2,25.0,686,Pangani,Manyara,250,2009,gravity,vwc,user-group,per bucket,good,enough,dam,surface,communal standpipe,functional,True,above,True,True
3,0.0,263,Ruvuma / Southern Coast,Mtwara,58,1986,submersible,vwc,user-group,never pay,good,dry,borehole,groundwater,communal standpipe,non functional/needs repair,True,above,False,True
4,0.0,0,Lake Victoria,Kagera,0,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional,False,level,False,False


In [185]:
!ls ../../data

features.csv
target.csv


In [187]:
#final_data.to_csv('../../data/clean_wells.csv')